# Majorana Operator 

In [1]:
from symred.symplectic_form import MajoranaOp
import numpy as np

# notes:

https://arxiv.org/pdf/2101.09349.pdf (pg11)

https://arxiv.org/abs/2110.10280

The Majorana operators {γ0, γ1, . . . , γm−1}, for m even, are linear Hermitian operators acting on the
fermionic Fock space 

$$H_{m/2} = \{ |b\rangle : b ∈ F_{2}^{m/2} \}$$

or equivalently the $m/2$-qubit complex Hilbert space satisfying $\forall 0 \leq i < j \leq m-1$:

1. $\gamma_{i}^{2} = \mathcal{I}$ - self inverse!
2. $\gamma_{i}\gamma_{j} = -\gamma_{j}\gamma_{i}$ - anti-commute!


These M single-mode operators generate a basis (up to phase factors) for the full algebra of Majorana operators via arbitrary products (https://arxiv.org/pdf/1908.08067.pdf pg9):

$$\gamma_{A} = \prod_{k \in A}^{M-1} \gamma_{k}$$

where $A \subseteq \{0,1,...,M-1 \} $ and represents the "support" of $\gamma_{A}$. We write this as $|A|$ where is the hamming weight of $\gamma_{A}$.

The anticommutator between two arbitrary Majorana operators $\gamma_{A}$ and $\gamma_{B}$ is determined by their individual supports and their overlap:



$$\{ \gamma_{A}, \gamma_{B} \} = \Big(1 + (-1)^{|A|\dot|B|- |A\cap B|} \Big) \gamma_{A}\gamma_{B}$$

Therefore:
- if $|A|\dot|B|- |A\cap B| = 0$
    - then terms anticommute
    
-else $|A|\dot|B|- |A\cap B| = 1$
    - and terms commute


# sympletic form

Class stores Majorana operator as a symplectic array and vector of coefficients.

rows of symplectic array give an individual operator and associated coefficient in coeff vec gives coefficient

therefore symplectic matrix is size $N \times M$ for $N$ terms and $M$ fermionic sites (note $M$ always even!)

In [2]:
operators = [
            [1,2,3,4], # op1
            [5] # op2
           ]
coeffs = [1,
         2]
Maj = MajoranaOp(operators, coeffs)
print(Maj)

(1+0j) γ1 γ2 γ3 γ4 +
(2+0j) γ5


In [3]:
Maj.symp_matrix

array([[0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 1]])

In [4]:
Maj.coeff_vec

array([1.+0.j, 2.+0.j])

# NOTE order matters!

code uses bubble sort to fix operator to normal form

In [5]:
ops = [
            [4,3]# op1
           ]
amps = [1]

Maj = MajoranaOp(ops, amps)
print(Maj)

# order flipped! generating a sign!

(-1+0j) γ3 γ4


# check commutation relations

uses above definition!

### 1. termwise commutation!

In [6]:
operators = [
            [1,2,3,4], # op1
            [5,6,10] # op2
           ]
coeffs = [1,
         2]
Maj1 = MajoranaOp(operators, coeffs)


operators2 = [
            [0], # op1
            [2], # op2
            [3], # op3
           ]
coeffs2 = [1,
           2,
           3+1j]
Maj2 = MajoranaOp(operators2, coeffs2)

In [7]:
Maj1.commutes_termwise(Maj2)

array([[1, 0, 0],
       [0, 0, 0]])

In [8]:
print(Maj1)
# can see no overlap of terms therefore do (4*3)%2 = 0... therefore terms anticommute!
Maj1.adjacency_matrix()

(1+0j) γ1 γ2 γ3 γ4 +
(2+0j) γ5 γ6 γ10


array([[1, 1],
       [1, 1]])

In [9]:
print(Maj2)
# can see no overlap of terms therefore do (1*1)%2 = 1... therefore terms commute!
Maj2.commutes_termwise(Maj2)

(1+0j) γ0 +
(2+0j) γ2 +
(3+1j) γ3


array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [10]:
operator = [
    [0,1,2,3],
    [5,6],
    [3,5],
    [0,1,2,3,4,5,6,7],
    [2,3],
    [4]
]

coeffs = np.arange(2,len(operator)+2)

###
M = MajoranaOp(operator, coeffs)
print(M)

M.adjacency_matrix()

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4


array([[1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 1, 1]])

In [11]:
print(M*M)
print()

C = M.to_OF_op()
out = C*C
print(out)

(17+0j) I +
(42+0j) γ4 γ5 γ6 +
(20+0j) γ4 γ5 γ6 γ7 +
(-56+0j) γ3 γ4 γ5 +
(36+0j) γ2 γ3 γ5 γ6 +
(84+0j) γ2 γ3 γ4 +
(-24+0j) γ0 γ1 +
(-60+0j) γ0 γ1 γ4 γ5 γ6 γ7 +
(40+0j) γ0 γ1 γ2 γ4 γ6 γ7 +
(12+0j) γ0 γ1 γ2 γ3 γ5 γ6 +
(28+0j) γ0 γ1 γ2 γ3 γ4 +
(-30+0j) γ0 γ1 γ2 γ3 γ4 γ7

(17+0j) () +
(-24+0j) (0, 1) +
(28+0j) (0, 1, 2, 3, 4) +
(-30+0j) (0, 1, 2, 3, 4, 7) +
(12+0j) (0, 1, 2, 3, 5, 6) +
(40+0j) (0, 1, 2, 4, 6, 7) +
(-60+0j) (0, 1, 4, 5, 6, 7) +
(84+0j) (2, 3, 4) +
(36+0j) (2, 3, 5, 6) +
(-56+0j) (3, 4, 5) +
(42+0j) (4, 5, 6) +
(20+0j) (4, 5, 6, 7)


In [12]:
M.adjacency_matrix()

array([[1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 1, 1]])

# convert from Fermions to Majoranas

In [13]:
from openfermion import FermionOperator, get_majorana_operator
from symred.symplectic_form import convert_openF_fermionic_op_to_maj_op

ham = (FermionOperator('0^ 3', .5) +
       FermionOperator('3^ 0', 0.5) +
      FermionOperator('3^ 2^ 0 1', 0.5))

M_out = convert_openF_fermionic_op_to_maj_op(ham)

print(M_out.to_OF_op() == get_majorana_operator(ham))

True


# get basis for operator

In [14]:
from symred.utils import gf2_basis_for_gf2_rref, gf2_gaus_elim
import os
import json
from openfermion import reverse_jordan_wigner, jordan_wigner
from symred.symplectic_form import PauliwordOp
from functools import reduce

In [15]:
working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir)
data_dir = os.path.join(parent_dir,'data')



file = 'O1_STO-3G_triplet_OO.json'
# file = 'H2-Be1_STO-3G_singlet_BeH2BeH2.json'

file_path = os.path.join(data_dir, file)
with open(file_path, 'r') as input_file:
    ham_dict = json.load(input_file)
    
ham = PauliwordOp(ham_dict)

ham_opemF = ham.PauliwordOp_to_OF

H_qubit = reduce(lambda x,y: x+y, ham_opemF)

fermionic_H = reverse_jordan_wigner(H_qubit)
maj_H = convert_openF_fermionic_op_to_maj_op(fermionic_H)

In [30]:
ZX_symp = maj_H.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

array([[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]])

In [31]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19


In [18]:
openF_M_op = maj_H.to_OF_op()
basis_op_openF =  basis_op.to_OF_op()
print('commmutes: ', openF_M_op*basis_op_openF == basis_op_openF*openF_M_op)

# print(maj_H.commutes_termwise(basis_op))
maj_H.commutes(basis_op)

commmutes:  True


True

In [19]:
jordan_wigner(basis_op.to_OF_op())

1j [Z0 Z3 Z5 Z7 Z9] +
1j [Z1 Z3 Z5 Z7 Z9] +
(-1+0j) [Z2 Z3] +
(-1+0j) [Z4 Z5] +
(1+0j) [Z6 Z7 Z8 Z9]

## Get cliffords to map to terms!

Section C of https://arxiv.org/pdf/2110.10280.pdf

A Majorana fermion stabilizer code [32], or Majorana stabilizer code for brevity, is the simultaneous +1 eigenspace of a collection of commuting, Hermitian, even **weight Majorana operators**. The evenness constraint ensures that these operators are fermion-parity preserving, and hence physically observable. 

Therefore rotate down onto pairs of majorana modes!

In [20]:
# even_inds = np.arange(0,basis_op.n_sites, 2) # 2i positions
# odd_inds = np.arange(1,basis_op.n_sites, 2) # 2i+1 positions

basis_op.symp_matrix[:, basis_op.even_inds]
basis_op.symp_matrix[:, basis_op.odd_inds]

array([[1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])

In [21]:
print(basis_op)

(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19


In [22]:
# remember consider pairs of sites!

pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds],
                      basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

row_sum = np.einsum('ij->i',pairs)
sqp_indices = np.where(pairs[np.where(row_sum==1)])[1]

non_sqp_basis = MajoranaOp(basis_op.symp_matrix[np.where(row_sum!=1)],
                            basis_op.coeff_vec[np.where(row_sum!=1)])

# _recursive_rotate_onto_sqp(non_sqp_basis)

# rotated_op = self.recursive_rotate_by_Pword(rotations)


In [23]:
print(basis_op)

(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19


In [24]:

n =  MajoranaOp([[5]], [1])
# n =  MajoranaOp([[0,1,2,3,4]], [1])

basis_op = n
print(basis_op)

print(jordan_wigner(n.to_OF_op()))

(1+0j) γ5
(1+0j) [Z0 Z1 Y2]


In [25]:
n = MajoranaOp([np.where(pivot_row!=0)[0]], [1])
# n =  MajoranaOp([[3,4,5,6]], [1])

basis_op = n
print(basis_op)

print(jordan_wigner(n.to_OF_op()))

NameError: name 'pivot_row' is not defined

In [ ]:
np.where(pivot_row!=0)

In [ ]:
basis_op.symp_matrix[:, basis_op.even_inds]
basis_op.symp_matrix[:, basis_op.odd_inds]

basis_op.odd_inds

In [ ]:
jordan_wigner(MajoranaOp([[0,1,2,3,4,5,6]], [1]).to_OF_op())

In [ ]:
jordan_wigner(MajoranaOp([np.arange(0,pivot_point_odd)], [1]).to_OF_op())

In [ ]:
# remember consider pairs of sites!
used_indices=[]

mode_pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds],
                      basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

# sum along pairs of indices to find how large a given term is!
row_sum = np.einsum('ij->i',mode_pairs)
# sort by least dense majorana operators
sort_rows_by_weight = np.argsort(row_sum)

# take first term (which is least dense)
pivot_row = basis_op.symp_matrix[sort_rows_by_weight][0]

non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices)) #<-checked

# gives positions in whole operator where single mode terms terms occur
col_sum = np.einsum('ij->j',basis_op.symp_matrix)

# pivot_row * col_sum : gives positions where pivot term and where other single mode terms occur
support = pivot_row*col_sum

# want to take lowest index support (as this will allow unique terms)
pivot_point_even = non_I[np.argmin(support[non_I])]
pivot_point_odd = pivot_point_even + 1 # 2i+1 index!


print(MajoranaOp([np.where(pivot_row!=0)[0]], [1]))
print(jordan_wigner(MajoranaOp([np.where(pivot_row!=0)[0]], [1]).to_OF_op()))

# first need to make pivot position non-diagonal!

# take term we are rotating to single term
rot_op = pivot_row.copy()
pivot_maj = MajoranaOp([np.where(pivot_row!=0)[0]], [1])

# check if pivot term is Pauli Z term... if so then rotate pivot position to something else!
Z_rot= False
if (pivot_row[pivot_point_even] + pivot_row[pivot_point_odd])==2:
    Z_rot = True
    # like Pauli X on pivot position
    rot_op_inds = np.arange(0,pivot_point_odd)
elif rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 0:
    raise ValueError('pivot error')

phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
if Z_rot:
    
    # number of Zs in rotation
    phase =(1j)**((len(rot_op_inds))//2) 
    
    maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])

    rot_basis_out = (maj_rot_op * basis_op * maj_rot_op_conj).cleanup()
    rot_piv =  (maj_rot_op * pivot_maj * maj_rot_op_conj).cleanup() 
else:
    rot_basis_out =  basis_op
    rot_piv = pivot_maj
    
print(rot_basis_out)
print(jordan_wigner(rot_basis_out.to_OF_op()))

# put correct Pauli on pivot position
rot_op2 = rot_piv.symp_matrix[0]
rot_op2[pivot_point_even] = (rot_op2[pivot_point_even]+1)%2
rot_op2[pivot_point_odd] = (rot_op2[pivot_point_odd]+1)%2
# print(jordan_wigner(MajoranaOp([np.where(rot_op2!=0)[0]], [1]).to_OF_op()))
phase2 =(1j)**(sum(np.logical_and(rot_op2[basis_op.even_inds],
                                 rot_op2[basis_op.odd_inds])))
rot_op2_inds = np.where(rot_op2!=0)[0]
maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), phase_conv[phase2]*np.sin(np.pi/4)])
maj_rot_op2_conj = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), -1*phase_conv[phase2]*np.sin(np.pi/4)])

rot_basis_out2 = (maj_rot_op2 * rot_basis_out * maj_rot_op2_conj).cleanup()
print()
print(jordan_wigner(rot_basis_out2.to_OF_op()))

In [ ]:
print(jordan_wigner(rot_basis_out.to_OF_op()))


In [ ]:
pivot_point_even

In [ ]:
pivot_point_even = 6
pivot_point_odd = 7

rot_op2 = rot_basis_out.symp_matrix[0].copy()
rot_op2

In [ ]:
# rot_op2 = rot_piv.symp_matrix[0].copy()
rot_op2 = rot_basis_out.symp_matrix[0].copy()
if (rot_op2[pivot_point_even]+rot_op2[pivot_point_odd])%2==1:
    # pivot position is X or Y
    # if X use Y and if Y use X to make pauli Z on this position
    rot_op2[pivot_point_even] = (rot_op2[pivot_point_even]+1)%2
    rot_op2[pivot_point_odd] = (rot_op2[pivot_point_odd]+1)%2
else:
    # pivot position is Pauli Z
    # find where non diag term is:
    print('lol')
    diag_inds = np.logical_xor(rot_op2[rot_basis_out.odd_inds],
                              rot_op2[rot_basis_out.even_inds]).astype(int)
    first_nondiag_ind = np.where(diag_inds!=0)[0][0]

    rot_op2[first_nondiag_ind] = (rot_op2[first_nondiag_ind]+1)%2
    rot_op2[first_nondiag_ind+1] = (rot_op2[first_nondiag_ind+1]+1)%2 # odd ind

    # zero out Z pivot pos
    rot_op2[pivot_point_even] = 0
    rot_op2[pivot_point_odd]  = 0

rot_op2_inds = np.where(rot_op2!=0)[0]
maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), phase_conv[phase2]*np.sin(np.pi/4)])
jordan_wigner(maj_rot_op2.to_OF_op())

In [ ]:
jordan_wigner(MajoranaOp([[2,3,4,5,6,7,8,10]], [1]).to_OF_op())

In [ ]:
print(rot_basis_out)

In [ ]:
rot_basis_out.symp_matrix

In [ ]:
mode_pairs

In [ ]:
print(basis_op)

In [ ]:
jordan_wigner(test.to_OF_op())

In [188]:
test = MajoranaOp([[0,1,2,3], [4,5]],[1,1])
print(jordan_wigner(test.to_OF_op()))
print()

out = majorana_rotations(test)
terms, rots = out.get_rotations()
print('###')
for t in terms:
    print(jordan_wigner(t.to_OF_op()))

(-1+0j) [Z0 Z1] +
1j [Z2]

(1+0j) γ4 γ5
###
1j [Z2]
(1+0j) [Z0]


In [200]:
class majorana_rotations():
    def __init__(self, majorana_basis):
        self.phase_conv_dict = {1: 1j, -1: -1j, 1j:1, -1j:-1}
        self.majorana_basis = majorana_basis
        
        self.used_indices  = None
        self.maj_rotations = None
        self.rotated_basis = None
        
    def get_rotations(self):
        self.used_indices=[]
        self.maj_rotations=[]
#         self.rotated_basis = []
        
        ## remove any single terms 
        maj_basis = self.majorana_basis.cleanup().copy()
        
        
        single_terms_rows = np.where(np.einsum('ij->i', maj_basis.symp_matrix)==2)[0]
        if single_terms_rows.shape[0]>0:
            z_term_check = np.logical_xor(maj_basis.symp_matrix[single_terms_rows][:, maj_basis.even_inds],
                                     maj_basis.symp_matrix[single_terms_rows][:, maj_basis.odd_inds]).astype(int)

            z_terms = np.where(np.einsum('ij->i', z_term_check)==0)[0]
            single_z_inds = single_terms_rows[z_terms]
        
        
        self.used_indices = np.union1d(self.used_indices, single_z_inds).astype(int).tolist()
        
        # append aleady single terms to rotated basis (that keeps tracks of single terms)
        self.rotated_basis = [MajoranaOp([np.where(sym_vec)[0]], [sym_coeff]) \
                              for sym_vec, sym_coeff in zip(maj_basis.symp_matrix[single_z_inds], maj_basis.coeff_vec[single_z_inds])]
        print(self.rotated_basis[0])
        
        
        # remove single Z terms then find rotaions
        maj_basis = MajoranaOp(np.delete(maj_basis.symp_matrix, single_z_inds, axis=0),
                              np.delete(maj_basis.coeff_vec, single_z_inds, axis=0))

        final_terms = self._recursively_rotate(maj_basis)
        return self.rotated_basis, self.maj_rotations
        
    def _recursively_rotate(self, maj_basis):
            
        single_terms_rows = np.where(np.einsum('ij->i', maj_basis.symp_matrix)==2)[0]
        if single_terms_rows.shape[0]>0:
            z_term_check = np.logical_xor(maj_basis.symp_matrix[single_terms_rows][:, maj_basis.even_inds],
                                          maj_basis.symp_matrix[single_terms_rows][:, maj_basis.odd_inds]
                                         ).astype(int)

            z_terms = np.where(np.einsum('ij->i', z_term_check)==0)[0]
            single_z_inds = single_terms_rows[z_terms]
        
            if len(single_z_inds) == maj_basis.n_terms:
                # all single qubit pauli Z's
                # break out of recursion!
                return maj_basis
            
            # mask out single Z terms
            maj_basis = MajoranaOp(np.delete(maj_basis.symp_matrix, single_z_inds, axis=0),
                                  np.delete(maj_basis.coeff_vec, single_z_inds, axis=0))
            seen_inds = np.union1d(self.used_indices, single_z_inds).astype(int).tolist()
            for ind in seen_inds:
                self.used_indices.append(ind)
        
        #### NOW ROTATE ONTO SINGLE TERM!
        mode_pairs = np.logical_or(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                   maj_basis.symp_matrix[:, maj_basis.odd_inds]
                                  ).astype(int)
        
        # sum along pairs of indices to find how large a given term is!
        row_sum = np.einsum('ij->i',mode_pairs)
        
        # sort by least dense majorana operators
        sort_rows_by_weight = np.argsort(row_sum)
        
        # take first term (which is least dense)
        pivot_row = maj_basis.symp_matrix[sort_rows_by_weight][0]
        
        # get non identity positions (also ignores positions that have been rotated onto!)
        non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(self.used_indices))
        
        # gives positions in whole operator where single mode terms terms occur
        col_sum = np.einsum('ij->j',maj_basis.symp_matrix)
        
        # pivot_row * col_sum : gives positions where pivot term and where other single mode terms occur
        support = pivot_row*col_sum
        
        # want to take lowest index support (as this will allow unique terms)
        pivot_point = non_I[np.argmin(support[non_I])]
        if pivot_point%2==0:
            pivot_point_even = pivot_point
            pivot_point_odd = pivot_point_even + 1 # 2i+1 index!
        else:
            pivot_point_odd  = pivot_point
            pivot_point_even = pivot_point_odd-1
        
        # take term we are rotating to single term
        rot_op = pivot_row.copy()
        pivot_maj = MajoranaOp([np.where(pivot_row!=0)[0]], [1])

        # check if pivot term is Pauli Z term... if so then rotate pivot position to something else!
        Z_rot= False
        if (pivot_row[pivot_point_even] + pivot_row[pivot_point_odd])==2:
            Z_rot = True
            # apply op like Pauli X on pivot position
            rot_op_inds = np.arange(0,pivot_point_odd)
        elif rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 0:
            raise ValueError('pivot error')
        
        if Z_rot:
            # number of Zs in rotation (majorana have complex phase here)
            phase =(1j)**((len(rot_op_inds))//2) 

            maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), self.phase_conv_dict[phase]*np.sin(np.pi/4)])
            maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* self.phase_conv_dict[phase]* np.sin(np.pi/4)])

            rot_basis_out = (maj_rot_op * maj_basis * maj_rot_op_conj).cleanup()
            rot_piv =  (maj_rot_op * pivot_maj * maj_rot_op_conj).cleanup() 
            
            self.maj_rotations.append(maj_rot_op)
        else:
            rot_basis_out =  maj_basis
            rot_piv = pivot_maj
                
        
        rot_op2 = rot_piv.symp_matrix[0].copy()
        # pivot position is X or Y
        # if X use Y and if Y use X to make pauli Z on this position
        rot_op2[pivot_point_even] = (rot_op2[pivot_point_even]+1)%2
        rot_op2[pivot_point_odd] = (rot_op2[pivot_point_odd]+1)%2

            
        # print(jordan_wigner(MajoranaOp([np.where(rot_op2!=0)[0]], [1]).to_OF_op()))
        phase2 =(1j)**(sum(np.logical_and(rot_op2[rot_piv.even_inds],
                                          rot_op2[rot_piv.odd_inds])))
        rot_op2_inds = np.where(rot_op2!=0)[0]
        maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), self.phase_conv_dict[phase2]*np.sin(np.pi/4)])
        maj_rot_op2_conj = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), -1*self.phase_conv_dict[phase2]*np.sin(np.pi/4)])

        rot_basis_out2 = (maj_rot_op2 * rot_basis_out * maj_rot_op2_conj).cleanup()
        
        self.maj_rotations.append(maj_rot_op2)
        
        rotated_term = (maj_rot_op2 * rot_piv * maj_rot_op2_conj).cleanup()
        self.rotated_basis.append(rotated_term)
        
        self.used_indices.append(pivot_point_even)
        self.used_indices.append(pivot_point_odd)
        
        
        return self._recursively_rotate(rot_basis_out2)
        

In [201]:
basis_op = MajoranaOp([
                      [1,2,3],
                      [5],
                      [0,1,2,3],
                      [2,3],
                      [4,5],
                      [1,4]], [1,1,1,1,1,1])
jordan_wigner(basis_op.to_OF_op())

1j [X0 Z1 X2] +
1j [Y0 Z1] +
(-1+0j) [Z0 Z1] +
(1+0j) [Z0 Z1 Y2] +
1j [Z1] +
1j [Z2]

In [166]:
basis_op.symp_matrix[:, basis_op.even_inds]

array([[0, 1, 0],
       [0, 0, 0],
       [1, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [158]:
basis_op.symp_matrix[:, basis_op.odd_inds]

array([[1, 1, 0],
       [0, 0, 1],
       [1, 1, 0],
       [0, 1, 0]])

In [92]:
basis_op.symp_matrix

array([[0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1]])

array([3, 1, 4])

In [148]:
basis_op.symp_matrix

array([[0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 0, 0]])

In [154]:
single_terms_rows.shape

(0,)

In [175]:
basis_op.symp_matrix[single_terms_rows][:,basis_op.even_inds]

array([[0, 1, 0],
       [0, 0, 1],
       [0, 0, 1]])

array([[0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 1]])

In [103]:
Z_terms = np.logical_and(basis_op.symp_matrix[:, basis_op.even_inds],
                         basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

Z_terms

array([[0, 1, 0],
       [0, 0, 0]])

In [202]:
out = majorana_rotations(basis_op)
print(jordan_wigner(basis_op.to_OF_op()))
print()
print()
terms, rots = out.get_rotations()

for t in terms:
    print(jordan_wigner(t.to_OF_op()))

1j [X0 Z1 X2] +
1j [Y0 Z1] +
(-1+0j) [Z0 Z1] +
(1+0j) [Z0 Z1 Y2] +
1j [Z1] +
1j [Z2]


(1+0j) γ4 γ5


ValueError: attempt to get argmin of an empty sequence

In [82]:
rot_maj_H = basis_op.copy()
for rot in rots[::-1]:
    rot_maj_H = (rot * rot_maj_H * rot.conjugate).cleanup()
    
    print(rot_maj_H)

1j γ4 γ5 +
-1j γ1 γ2 γ3 γ4


In [38]:
rot_maj_H = maj_H.copy()
for rot in rots:
    rot_maj_H = (rot * rot_maj_H * rot.conjugate).cleanup()
    
jordan_wigner(rot_maj_H.to_OF_op())

0.011861111279596047j [Z0 Z1 X2 Y3 X4 X6 Z8] +
0.011861111279596047j [Z0 Z1 X2 X4 Y5 X6 Z8] +
(-0.00896331655087819+0j) [Z0 Z1 X2 X4 X6 Y7 X8] +
0.04309359338016289j [Z0 Z1 X2 X4 X6 Y7 Z9] +
(-0.006456886215848404+0j) [Z0 Z1 X2 X4 X6 Y8 X9] +
0.22003977334376137j [Z0 Z1 X2 X4 X6 Z8] +
-0.00896331655087819j [Z0 Z1 X2 X4 X6 Z8 Y9] +
0.011861111279596047j [Z0 X2 Y3 X4 Z5 X6 Z7 Z8 Z9] +
0.011861111279596047j [Z0 X2 Z3 X4 Y5 X6 Z7 Z8 Z9] +
-0.0034451672284630305j [Z0 X2 Z3 X4 Z5 X6 X7 X9] +
0.04309359338016289j [Z0 X2 Z3 X4 Z5 X6 Y7] +
(-0.00896331655087819+0j) [Z0 X2 Z3 X4 Z5 X6 Y7 X8 Z9] +
-0.0034451672284630305j [Z0 X2 Z3 X4 Z5 X6 Y7 Y9] +
(0.01240848377934122+0j) [Z0 X2 Z3 X4 Z5 X6 Z7 X8] +
(-0.006456886215848404+0j) [Z0 X2 Z3 X4 Z5 X6 Z7 Y8 X9] +
-0.278848316045859j [Z0 X2 Z3 X4 Z5 X6 Z7 Z8] +
-0.00896331655087819j [Z0 X2 Z3 X4 Z5 X6 Z7 Z8 Y9] +
-1.3339095688459306j [Z0 X2 Z3 X4 Z5 X6 Z7 Z8 Z9] +
-0.27239142983001063j [Z0 X2 Z3 X4 Z5 X6 Z7 Z9] +
-0.20085914446810477j [Z0 X2 Z3 X4 Z5 X6

In [ ]:
asdf

In [ ]:
ZX_symp = maj_H.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

In [ ]:
# remember consider pairs of sites!
used_indices=[]

mode_pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds],
                      basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

# sum along pairs of indices to find how large a given term is!
row_sum = np.einsum('ij->i',mode_pairs)
# sort by least dense majorana operators
sort_rows_by_weight = np.argsort(row_sum)

# take first term (which is least dense)
pivot_row = basis_op.symp_matrix[sort_rows_by_weight][0]

non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices)) #<-checked

# gives positions in whole operator where single mode terms terms occur
col_sum = np.einsum('ij->j',basis_op.symp_matrix)

# pivot_row * col_sum : gives positions where pivot term and where other single mode terms occur
support = pivot_row*col_sum

# want to take lowest index support (as this will allow unique terms)
pivot_point_even = non_I[np.argmin(support[non_I])]
pivot_point_odd = pivot_point_even + 1 # 2i+1 index!


# take term we are rotating to single term
rot_op = pivot_row.copy()

# check if "Pauli Z" anywhere... if so then rotate away!
Z_rot = False
if rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 1:
    Z_rot = True
    rot_op[pivot_point_odd]=0
elif rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 0:
    raise ValueError('pivot error')

    
if Z_rot:
    phase =(1j)**(sum(np.logical_and(rot_op[basis_op.even_inds],
                                     rot_op[basis_op.odd_inds])))

    # print(phase)

    phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}

    rot_op_inds = np.where(rot_op!=0)[0]
    maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])

    rot_basis_out = (maj_rot_op * basis_op * maj_rot_op_conj).cleanup()
    # print(rot_basis_out)
else:
    rot_basis_out = basis_op 
    
rot_op2 = np.zeros_like(rot_op)
rot_op2[:pivot_point_odd] = (rot_op[:pivot_point_odd])%2

phase2 =(1j)**(sum(np.logical_and(rot_op2[basis_op.even_inds],
                                 rot_op2[basis_op.odd_inds])))

rot_op2_inds = np.where(rot_op2!=0)[0]
maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), phase_conv[phase2]*np.sin(np.pi/4)])
maj_rot_op2_conj = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), -1*phase_conv[phase2]*np.sin(np.pi/4)])

rot_basis_out2 = (maj_rot_op2 * rot_basis_out * maj_rot_op2_conj).cleanup()

print(rot_basis_out2)

In [ ]:
# remember consider pairs of sites!
used_indices=[]

mode_pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds],
                      basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

# sum along pairs of indices to find how large a given term is!
row_sum = np.einsum('ij->i',mode_pairs)
# sort by least dense majorana operators
sort_rows_by_weight = np.argsort(row_sum)

# take first term (which is least dense)
pivot_row = basis_op.symp_matrix[sort_rows_by_weight][0]

non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))

# gives positions in whole operator where single mode terms terms occur
col_sum = np.einsum('ij->j',basis_op.symp_matrix)

# pivot_row * col_sum : gives positions where pivot term and where other single mode terms occur
support = pivot_row*col_sum

# want to take lowest index support (as this will allow unique terms)
pivot_point_even = non_I[np.argmin(support[non_I])]
pivot_point_odd = pivot_point_even + 1 # 2i+1 index!


# take term we are rotating to single term
rot_op = pivot_row.copy()

# check if "Pauli Z" anywhere... if so then rotate away!
Z_rot = False
if rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 1:
    Z_rot = True
    rot_op[pivot_point_odd]=0
elif rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 0:
    raise ValueError('pivot error')

    
if Z_rot:
    phase =(1j)**(sum(np.logical_and(rot_op[basis_op.even_inds],
                                     rot_op[basis_op.odd_inds])))

    # print(phase)

    phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}

    rot_op_inds = np.where(rot_op!=0)[0]
    maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])

    rot_basis_out = (maj_rot_op * basis_op * maj_rot_op_conj).cleanup()
    # print(rot_basis_out)
else:
    rot_basis_out = basis_op 
    
rot_op2 = np.zeros_like(rot_op)
rot_op2[:pivot_point_odd] = (rot_op[:pivot_point_odd])%2

phase2 =(1j)**(sum(np.logical_and(rot_op2[basis_op.even_inds],
                                 rot_op2[basis_op.odd_inds])))

rot_op2_inds = np.where(rot_op2!=0)[0]
maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), phase_conv[phase2]*np.sin(np.pi/4)])
maj_rot_op2_conj = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), -1*phase_conv[phase2]*np.sin(np.pi/4)])

rot_basis_out2 = (maj_rot_op2 * rot_basis_out * maj_rot_op2_conj).cleanup()

print(rot_basis_out2)

In [ ]:
df

In [ ]:
rot_op2

In [ ]:
print(maj_rot_op2)

In [ ]:
print(rot_basis_out2)

In [ ]:
rot_basis_out
jordan_wigner(rot_basis_out.to_OF_op())

In [ ]:
v = MajoranaOp([[5]],[1])
b = MajoranaOp([[0,1,2,3]],[1])
jordan_wigner(b.to_OF_op())

In [ ]:
pivot_point_even

In [ ]:
Z0 Z1 X2

In [ ]:
jordan_wigner(rot_basis_out.to_OF_op())

In [ ]:
print(basis_op)

In [ ]:
    # find rows of mode_pairs matrix that are not already unique
    free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)

    ## use integar sort to sort non_unique rows
    int_list = mode_pairs[:, free_mode_inds] @ (1 << np.arange(mode_pairs[:, free_mode_inds].shape[1])[::-1])
    sort_rows_by_weight = np.argsort(-1*int_list)

    ## find indpendent index of pivot! (cannot use dependent term)
    # sum down column... note any term with 1 in row is unique term
    support = np.einsum('ij->j', mode_pairs)
    support[unqiue_mode_indices]=2 # push any unique index to above 1 (so as not to use)
    pivot_index_even = 2*np.where(support==1)[0][0] # gives independent index for symp matrix (not times 2 as 2i not i)
    pivot_index_odd = pivot_index_even + 1 # get odd pivot index too
    
    # take term we are rotating to single term
    rot_op = basis.symp_matrix[sort_rows_by_weight][0].copy()
    
    print('reduce:', jordan_wigner(MajoranaOp(np.array([rot_op]), [1]).to_OF_op()))
    if rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 1:
        rot_op[pivot_index_odd]=0
    elif rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 0:
        raise ValueError('pivot error')

In [ ]:
row_sum = np.aeinsum('ij->i',basis.symp_matrix)
col_sum = np.einsum('ij->j',basis.symp_matrix)
sort_rows_by_weight = np.argsort(row_sum)
pivot_row = basis.symp_matrix[sort_rows_by_weight][0]
non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
support = pivot_row*col_sum
pivot_point = non_I[np.argmin(support[non_I])]
pivot_rotation = update_sets(pivot_row.copy(), pivot_point)
rotated_basis = basis._rotate_by_single_Pword(pivot_rotation)
non_sqp = np.where(np.einsum('ij->i', rotated_basis.symp_matrix)!=1)[0].tolist()

In [ ]:
## for non pairs use individual
# op1 = MajoranaOp([[1,6,7,10,11,14,15,18,19]],[1])
# allowed_inds= [0,6,7,10,11,14,15,18,19]

# for pairs used even ind!
op1 = MajoranaOp([[0,1,6,7,10,11,14,15,18,19]],[1])
allowed_inds= [0,6,7,10,11,14,15,18,19]

rott = MajoranaOp([allowed_inds],[1])
phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
phase =(1j)**(sum(np.logical_and(new_op[rott.even_inds],
                                 new_op[rott.odd_inds])))

rott = MajoranaOp([[], allowed_inds] ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rott2 = MajoranaOp([[], allowed_inds],[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])

print(rott.commutes(op1))

print(rott * op1 * rott2)


# rot_op = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])

In [ ]:
basis = basis_op.copy()    

# check if unique (equiv to single qubit Pauli operators!)
mode_pairs = np.logical_or(basis.symp_matrix[:, basis.even_inds], 
                           basis.symp_matrix[:, basis.odd_inds]).astype(int)

mode_row_sum = np.einsum('ij->i',mode_pairs)
unique_rows = (mode_row_sum==1)

unqiue_mode_indices = np.where(np.einsum('ij->j',mode_pairs[unique_rows])==1)[0]


free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)
free_mode_inds

In [ ]:
from openfermion import jordan_wigner

In [ ]:
rot_basis = basis

In [ ]:
basis_op

In [ ]:
basis = basis_op.copy()    

# check if unique (equiv to single qubit Pauli operators!)
mode_pairs = np.logical_or(basis.symp_matrix[:, basis.even_inds], 
                           basis.symp_matrix[:, basis.odd_inds]).astype(int)

mode_row_sum = np.einsum('ij->i',mode_pairs)
unique_rows = (mode_row_sum==1)


if all(unique_rows):
    # all unique!
    pass
else:
    # sort basis by left most terms

    unqiue_mode_indices = np.where(np.einsum('ij->j',mode_pairs[unique_rows])==1)[0]

#         even_inds_used = np.sort(2*unqiue_mode_indices)
#         odd_inds_used = even_inds_used+1
#         used_inds = np.union1d(even_inds_used, odd_inds_used)
#         free_inds = np.setdiff1d(np.arange(basis.symp_matrix.shape[1]), used_inds)

    # find rows of mode_pairs matrix that are not already unique
    free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)

    ## use integar sort to sort non_unique rows
    int_list = mode_pairs[:, free_mode_inds] @ (1 << np.arange(mode_pairs[:, free_mode_inds].shape[1])[::-1])
    sort_rows_by_weight = np.argsort(-1*int_list)

    ## find indpendent index of pivot! (cannot use dependent term)
    # sum down column... note any term with 1 in row is unique term
    support = np.einsum('ij->j', mode_pairs)
    support[unqiue_mode_indices]=2 # push any unique index to above 1 (so as not to use)
    pivot_index_even = 2*np.where(support==1)[0][0] # gives independent index for symp matrix (not times 2 as 2i not i)
    pivot_index_odd = pivot_index_even + 1 # get odd pivot index too
    
    # take term we are rotating to single term
    rot_op = basis.symp_matrix[sort_rows_by_weight][0].copy()
    
#     print('reduce:', jordan_wigner(MajoranaOp(np.array([rot_op]), [1]).to_OF_op()))
    if rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 1:
        rot_op[pivot_index_odd]=0
    elif rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 0:
        raise ValueError('pivot error')
    
    rot_op[0]=1
    rot_op[1]=1
    
    phase =(1j)**(sum(np.logical_and(rot_op[basis_op.even_inds],
                                     rot_op[basis_op.odd_inds])))

    print(phase)
    
    phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}

    rot_op_inds = np.where(rot_op!=0)[0]
    maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])
#         rotations.append((str(MajoranaOp(np.array([pivot_row_rot]), [1]).__str__()), None))
    
#     print('using:', jordan_wigner(maj_rot_op.to_OF_op()))
#     print(maj_rot_op)
    rot_basis_out = maj_rot_op * basis * maj_rot_op_conj
    
#     print(maj_rot_op * MajoranaOp(np.array([basis.symp_matrix[sort_rows_by_weight][0]]),[1]) * maj_rot_op_conj)

print()
print(rot_basis_out)
print()
print(jordan_wigner(rot_basis_out.to_OF_op()))


In [ ]:
print(basis_op, '\n')
op1 = MajoranaOp([[],[0]],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op1_dag = MajoranaOp([[],[0]],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot1 = op1*basis_op*op1_dag
print(rot1)

op1.commutes_termwise(basis_op)

In [ ]:
print(op1*op1_dag)

In [ ]:
print(op1.commutes(basis_op))

op1.commutes_termwise(basis_op)

In [ ]:
print(rot1)
print(op2)

In [ ]:
op2 = MajoranaOp([[], [1, 6]],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op2_dag = MajoranaOp([[], [1, 6]],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot2 = op2*rot1*op2_dag
print(rot2)

rot1.commutes_termwise(pp)

In [ ]:
print(op1*basis_op*op1_dag)

In [ ]:
# bug fixed
op2 = MajoranaOp([[12, 10]], [1])
print(op2)
from openfermion import MajoranaOperator
test = MajoranaOperator(term=(12,10))
print(test)

In [ ]:
class majorana_rotations():
    def __init__(self, majorana_basis):
        self.phase_conv_dict = {1: 1j, -1: -1j, 1j:1, -1j:-1}
        self.majorana_basis = majorana_basis
        
        self.used_indices  = None
        self.maj_rotations = None
        self.rotated_basis = None
        
    def get_rotations(self):
        self.used_indices=[]
        self.maj_rotations=[]
        self.rotated_basis = []
        
        ## remove any single terms 
        maj_basis = self.majorana_basis.cleanup().copy()
        Z_terms = np.logical_and(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                 maj_basis.symp_matrix[:, maj_basis.odd_inds]).astype(int)
        
        Z_rows = np.einsum('ij->i',Z_terms)
        
        self.used_indices = np.union1d(self.used_indices, np.where(Z_rows==1)[0]).astype(int).tolist()
        print(maj_basis)
        print(self.used_indices)
        
        non_single_Z = np.where(Z_rows!=1)[0]
        maj_basis = MajoranaOp(maj_basis.symp_matrix[non_single_Z], maj_basis.coeff_vec[non_single_Z])
        
        print(maj_basis)
        
        final_terms = self._recursively_rotate(maj_basis)
        return self.rotated_basis, self.maj_rotations
        
    def _recursively_rotate(self, maj_basis):
        
        Z_terms = np.logical_and(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                 maj_basis.symp_matrix[:, maj_basis.odd_inds]).astype(int)
        
        Z_rows = np.einsum('ij->i',Z_terms)
        
        if sum(Z_rows) == maj_basis.n_terms:
            # all single qubit pauli Z's
            return maj_basis
        
        # mask out single Z terms
        non_single_Z = np.where(Z_rows!=1)[0]
        maj_basis = MajoranaOp(maj_basis.symp_matrix[non_single_Z], maj_basis.coeff_vec[non_single_Z])
        
        
        #### NOW ROTATE ONTO SINGLE TERM!
        mode_pairs = np.logical_or(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                   maj_basis.symp_matrix[:, maj_basis.odd_inds]
                                  ).astype(int)
        
        # sum along pairs of indices to find how large a given term is!
        row_sum = np.einsum('ij->i',mode_pairs)
        
        # sort by least dense majorana operators
        sort_rows_by_weight = np.argsort(row_sum)
        
        # take first term (which is least dense)
        pivot_row = maj_basis.symp_matrix[sort_rows_by_weight][0]
        
        # get non identity positions (also ignores positions that have been rotated onto!)
        non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(self.used_indices))
        
        # gives positions in whole operator where single mode terms terms occur
        col_sum = np.einsum('ij->j',maj_basis.symp_matrix)
        
        # pivot_row * col_sum : gives positions where pivot term and where other single mode terms occur
        support = pivot_row*col_sum
        
        # want to take lowest index support (as this will allow unique terms)
        pivot_point = non_I[np.argmin(support[non_I])]
        if pivot_point%2==0:
            pivot_point_even = pivot_point
            pivot_point_odd = pivot_point_even + 1 # 2i+1 index!
        else:
            pivot_point_odd = pivot_point
            pivot_point_even = pivot_point_odd-1
        
        # take term we are rotating to single term
        rot_op = pivot_row.copy()
        pivot_maj = MajoranaOp([np.where(pivot_row!=0)[0]], [1])

        # check if pivot term is Pauli Z term... if so then rotate pivot position to something else!
        Z_rot= False
        if (pivot_row[pivot_point_even] + pivot_row[pivot_point_odd])==2:
            Z_rot = True
            # apply op like Pauli X on pivot position
            rot_op_inds = np.arange(0,pivot_point_odd)
            print('inds:', rot_op_inds)
        elif rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 0:
            raise ValueError('pivot error')
        
        if Z_rot:
            # number of Zs in rotation (majorana have complex phase here)
            phase =(1j)**((len(rot_op_inds))//2) 

            maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), self.phase_conv_dict[phase]*np.sin(np.pi/4)])
            maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* self.phase_conv_dict[phase]* np.sin(np.pi/4)])

            rot_basis_out = (maj_rot_op * maj_basis * maj_rot_op_conj).cleanup()
            rot_piv =  (maj_rot_op * pivot_maj * maj_rot_op_conj).cleanup() 
            
            self.maj_rotations.append(maj_rot_op)
        else:
            rot_basis_out =  maj_basis
            rot_piv = pivot_maj
        
        print(maj_rot_op)
        
        
        rot_op2 = rot_piv.symp_matrix[0].copy()
        # pivot position is X or Y
        # if X use Y and if Y use X to make pauli Z on this position
        rot_op2[pivot_point_even] = (rot_op2[pivot_point_even]+1)%2
        rot_op2[pivot_point_odd] = (rot_op2[pivot_point_odd]+1)%2

            
        # print(jordan_wigner(MajoranaOp([np.where(rot_op2!=0)[0]], [1]).to_OF_op()))
        phase2 =(1j)**(sum(np.logical_and(rot_op2[rot_piv.even_inds],
                                          rot_op2[rot_piv.odd_inds])))
        rot_op2_inds = np.where(rot_op2!=0)[0]
        maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), phase_conv[phase2]*np.sin(np.pi/4)])
        maj_rot_op2_conj = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), -1*phase_conv[phase2]*np.sin(np.pi/4)])

        rot_basis_out2 = (maj_rot_op2 * rot_basis_out * maj_rot_op2_conj).cleanup()
        
        self.maj_rotations.append(maj_rot_op2)
        
        rotated_term = (maj_rot_op2 * rot_piv * maj_rot_op2_conj).cleanup()
        self.rotated_basis.append(rotated_term)
        
        self.used_indices.append(pivot_point_even)
        self.used_indices.append(pivot_point_odd)
        
        print(rot_basis_out2, 'LOL')
        print(rot_piv)
        
        return self._recursively_rotate(rot_basis_out2)
        

In [ ]:
class majorana_rotations():
    def __init__(self, majorana_basis):
        self.phase_conv_dict = {1: 1j, -1: -1j, 1j:1, -1j:-1}
        self.majorana_basis = majorana_basis
        
        self.used_indices  = None
        self.maj_rotations = None
        self.rotated_basis = None
        
    def get_rotations(self):
        self.used_indices=[]
        self.maj_rotations=[]
#         self.rotated_basis = []
        
        ## remove any single terms 
        maj_basis = self.majorana_basis.cleanup().copy()
        Z_terms = np.logical_and(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                 maj_basis.symp_matrix[:, maj_basis.odd_inds]).astype(int)
        
        Z_rows = np.einsum('ij->i',Z_terms)
        non_single_Z = np.where(Z_rows!=1)[0]
        single_Z = np.where(Z_rows==1)[0]
        
        self.used_indices = np.union1d(self.used_indices, single_Z).astype(int).tolist()
        
        # append aleady single terms to rotated basis (that keeps tracks of single terms)
        self.rotated_basis = [MajoranaOp([np.where(sym_vec)[0]], [sym_coeff]) \
                              for sym_vec, sym_coeff in zip(maj_basis.symp_matrix[single_Z], maj_basis.coeff_vec[single_Z])]
        print(self.rotated_basis[0])
        
        maj_comp = MajoranaOp(maj_basis.symp_matrix[non_single_Z], maj_basis.coeff_vec[non_single_Z])
        
        # remove single Z terms then find rotaions
        maj_basis = MajoranaOp(maj_basis.symp_matrix[non_single_Z], maj_basis.coeff_vec[non_single_Z])
        final_terms = self._recursively_rotate(maj_basis)
        return self.rotated_basis, self.maj_rotations
        
    def _recursively_rotate(self, maj_basis):
        
        Z_terms = np.logical_and(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                 maj_basis.symp_matrix[:, maj_basis.odd_inds]).astype(int)
        
        Z_rows = np.einsum('ij->i',Z_terms)
        
        if sum(Z_rows) == maj_basis.n_terms:
            # all single qubit pauli Z's
            return maj_basis
        
        # mask out single Z terms
        non_single_Z = np.where(Z_rows!=1)[0]
        maj_basis = MajoranaOp(maj_basis.symp_matrix[non_single_Z], maj_basis.coeff_vec[non_single_Z])
        
        
        #### NOW ROTATE ONTO SINGLE TERM!
        mode_pairs = np.logical_or(maj_basis.symp_matrix[:, maj_basis.even_inds],
                                   maj_basis.symp_matrix[:, maj_basis.odd_inds]
                                  ).astype(int)
        
        # sum along pairs of indices to find how large a given term is!
        row_sum = np.einsum('ij->i',mode_pairs)
        
        # sort by least dense majorana operators
        sort_rows_by_weight = np.argsort(row_sum)
        
        # take first term (which is least dense)
        pivot_row = maj_basis.symp_matrix[sort_rows_by_weight][0]
        
        # get non identity positions (also ignores positions that have been rotated onto!)
        non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(self.used_indices))
        
        # gives positions in whole operator where single mode terms terms occur
        col_sum = np.einsum('ij->j',maj_basis.symp_matrix)
        
        # pivot_row * col_sum : gives positions where pivot term and where other single mode terms occur
        support = pivot_row*col_sum
        
        # want to take lowest index support (as this will allow unique terms)
        pivot_point = non_I[np.argmin(support[non_I])]
        if pivot_point%2==0:
            pivot_point_even = pivot_point
            pivot_point_odd = pivot_point_even + 1 # 2i+1 index!
        else:
            pivot_point_odd  = pivot_point
            pivot_point_even = pivot_point_odd-1
        
        # take term we are rotating to single term
        rot_op = pivot_row.copy()
        pivot_maj = MajoranaOp([np.where(pivot_row!=0)[0]], [1])

        # check if pivot term is Pauli Z term... if so then rotate pivot position to something else!
        Z_rot= False
        if (pivot_row[pivot_point_even] + pivot_row[pivot_point_odd])==2:
            Z_rot = True
            # apply op like Pauli X on pivot position
            rot_op_inds = np.arange(0,pivot_point_odd)
        elif rot_op[pivot_point_even] == rot_op[pivot_point_odd] == 0:
            raise ValueError('pivot error')
        
        if Z_rot:
            # number of Zs in rotation (majorana have complex phase here)
            phase =(1j)**((len(rot_op_inds))//2) 

            maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), self.phase_conv_dict[phase]*np.sin(np.pi/4)])
            maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* self.phase_conv_dict[phase]* np.sin(np.pi/4)])

            rot_basis_out = (maj_rot_op * maj_basis * maj_rot_op_conj).cleanup()
            rot_piv =  (maj_rot_op * pivot_maj * maj_rot_op_conj).cleanup() 
            
            self.maj_rotations.append(maj_rot_op)
        else:
            rot_basis_out =  maj_basis
            rot_piv = pivot_maj
                
        
        rot_op2 = rot_piv.symp_matrix[0].copy()
        # pivot position is X or Y
        # if X use Y and if Y use X to make pauli Z on this position
        rot_op2[pivot_point_even] = (rot_op2[pivot_point_even]+1)%2
        rot_op2[pivot_point_odd] = (rot_op2[pivot_point_odd]+1)%2

            
        # print(jordan_wigner(MajoranaOp([np.where(rot_op2!=0)[0]], [1]).to_OF_op()))
        phase2 =(1j)**(sum(np.logical_and(rot_op2[rot_piv.even_inds],
                                          rot_op2[rot_piv.odd_inds])))
        rot_op2_inds = np.where(rot_op2!=0)[0]
        maj_rot_op2 = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), self.phase_conv_dict[phase2]*np.sin(np.pi/4)])
        maj_rot_op2_conj = MajoranaOp([[],rot_op2_inds], [np.cos(np.pi/4), -1*self.phase_conv_dict[phase2]*np.sin(np.pi/4)])

        rot_basis_out2 = (maj_rot_op2 * rot_basis_out * maj_rot_op2_conj).cleanup()
        
        self.maj_rotations.append(maj_rot_op2)
        
        rotated_term = (maj_rot_op2 * rot_piv * maj_rot_op2_conj).cleanup()
        self.rotated_basis.append(rotated_term)
        
        self.used_indices.append(pivot_point_even)
        self.used_indices.append(pivot_point_odd)
        
        return self._recursively_rotate(rot_basis_out2)
        